In [67]:
from selenium import webdriver
import urllib,re  
from selenium.webdriver.support.select import Select
import time     
from bs4 import BeautifulSoup
import pandas as pd
import PyPDF2 as pdf
import lxml
sleep_time=1

In [66]:
def pdf2text():
    pdfFileObj = open('Lawfile.pdf', 'rb')
    pdfReader = pdf.PdfFileReader(pdfFileObj)
    pagenum=pdfReader.numPages
    text=''
    for i in range(pagenum):
        pageObj = pdfReader.getPage(i)
        text+=' '+pageObj.extractText() 
    pdfFileObj.close()
    return text

In [65]:
def Home(name='Administration'):
    # uploading chromedrive file 
    chrome_options = webdriver.ChromeOptions()
    prefs = {"profile.default_content_setting_values.notifications" : 2}
    chrome_options.add_experimental_option("prefs",prefs)
    browser = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)

    # openning URL in chromedriver 
    URL='https://caselaw.findlaw.com/summary/search'
    browser.get(URL)
    browser.maximize_window()

    # Preparing Home page scrapeable 
#     time.sleep(sleep_time)
    browser.execute_script('window.scrollTo(0, document.body.scrollHeight / 4)')
    time.sleep(sleep_time)
    select=Select(browser.find_element_by_name('topic'))
    time.sleep(sleep_time)
    soup=BeautifulSoup(browser.page_source,'lxml')
    legal_name={i:[optiontag['value'],optiontag.text] for i,optiontag in enumerate(soup.find('select',attrs={'id':'topic'}).find_all('option')[1:])}
    print(legal_name)
    n=int(input('Select a LawName key in integer : '))
    select.select_by_value(legal_name[n][0])
    url=f'https://caselaw.findlaw.com/summary/search/?query=filters&dateFormat=yyyyMMdd&topic={legal_name[n][0]}&pgnum=1'
    browser.get(url)
    return browser

In [64]:
def parse_data(raw_html):
    label=raw_html.find('i').text
    desc=raw_html.find('div',attrs={'class':'desc_subttl'}).text
    link=raw_html.find('a')['href']
    date=raw_html.find_all('td')[-2].text
    return [desc,link,date,label]

def scrape_text(browser,url):
    try:
        browser.window_handles[1]
    except Exception as e:
        browser.execute_script("window.open('');")
    browser.switch_to.window(browser.window_handles[1])
    browser.get(url)
    status=browser.find_element_by_class_name('caselawcontent').text
    time.sleep(sleep_time)
    tinybtn=browser.find_element_by_link_text('READ').click()
    time.sleep(sleep_time)
    text=''
    try:
        herebtn=browser.find_element_by_link_text('here')
        herebtn.click()
        browser.switch_to.window(browser.window_handles[2])
        url=browser.current_url
        file = urllib.request.urlopen(url)
        pdffile = open('Lawfile.pdf', 'wb')
        pdffile.write(file.read())
        text=pdf2text()
    except Exception as e:
        try:
            text=browser.find_element_by_class_name('caselawcontent').text
        except Exception as e:
            print(e)
#     browser.close()
    browser.switch_to.window(browser.window_handles[1])
    return text,status
statusLabel=['affirmed','held','remanded','vacated','denied','reversed','dismissed','vacated and remanded',
             'revived','revised','vocated and affirmed','reverse,remanded,affirmed','affirmed,reversed',
             'reaffirmed','reanded,reversed','rejected']

def findStatus(text):
    text=text.lower()
    text=re.sub("[\W]",' ',text)
    print(text)
    sLabels =[sl for sl in statusLabel if text.find(sl)>=0]
    if len(sLabels)>1:
        return ' and '.join([slr for slr in set(sl for l in sLabels for sl in l.split()) if slr in statusLabel])
    return sLabels[-1]

def ScrapeLinksAndSaveData(browser):
    global counter
    soup=BeautifulSoup(browser.page_source,'lxml')
    fileLinks=soup.find_all('tr')
    for i in range(1,len(fileLinks)):
        trlst=parse_data(fileLinks[i])
        text,status=scrape_text(browser,trlst[1])
        status=findStatus(status)
        trlst.extend([text,status])
        df.loc[counter]=trlst
        counter+=1
    if len(df)%10==0:
        df.to_csv('Law.csv')
        print('Check point created at dataframe length :  {}'.format(len(df)))
        

In [68]:
df=pd.DataFrame(columns=['Description','Url','Date','Label','Text','Status'])
counter=0
browser=Home()
totalPages=int(browser.find_element_by_class_name('pagination').text.split('\n')[-2])
pages=1

{0: ['cs_1', 'Administrative Law'], 1: ['cs_2', 'Admiralty'], 2: ['cs_3', 'Agriculture'], 3: ['cs_4', 'Antitrust & Trade Regulation'], 4: ['cs_79', 'Asset Forfeiture'], 5: ['cs_5', "Attorney's Fees"], 6: ['cs_6', 'Banking Law'], 7: ['cs_7', 'Bankruptcy Law'], 8: ['cs_8', 'Civil Procedure'], 9: ['cs_9', 'Civil Rights'], 10: ['cs_10', 'Class Actions'], 11: ['cs_11', 'Commercial Law'], 12: ['cs_12', 'Communications Law'], 13: ['cs_13', 'Constitutional Law'], 14: ['cs_14', 'Construction'], 15: ['cs_15', 'Consumer Protection Law'], 16: ['cs_16', 'Contracts'], 17: ['cs_17', 'Corp. Governance'], 18: ['cs_18', 'Corporation & Enterprise Law'], 19: ['cs_19', 'Criminal Law & Procedure'], 20: ['cs_20', 'Habeas Corpus'], 21: ['cs_21', 'Sentencing'], 22: ['cs_22', 'Cyberspace Law'], 23: ['cs_23', 'Debt Collection'], 24: ['cs_24', 'Dispute Resolution & Arbitration'], 25: ['cs_25', 'Drugs & Biotech'], 26: ['cs_31', 'ERISA'], 27: ['cs_26', 'Education Law'], 28: ['cs_27', 'Elder Law'], 29: ['cs_28', 'El

In [69]:

while pages!=totalPages:
    try:
        ScrapeLinksAndSaveData(browser)
    except Exception as e:
        print(e)
    browser.switch_to.window(browser.window_handles[0])
    nexturl=browser.current_url
    nexturl=nexturl.split('pgnum=')
    nexturl=nexturl[0]+'pgnum='+str(int(nexturl[1])+1)
    browser.get(nexturl)
    try:
        browser.execute_script('window.scrollTo(0, document.body.scrollHeight)/4')
    except Exception as e:
        pass
    pages+=1

lockett v  bonson  19 1012 read lockett v  bonson  19 1012 read affirmed  the grant of a defense motion for summary judgement in an eighth amendment claim brought by a prisoner against prison nurses he says were deliberately indifferent to his needs as a sickle cell disease sufferer was proper because he failed to exhaust his administrative remedies against one nurse and the record wouldn t support a jury finding on his claim against the other  appellate information decided published 2019 08 28 judges ripple court united states seventh circuit counsel
chaidez v  ford motor company  18 2753 read chaidez v  ford motor company  18 2753 read vacated and remanded  the district court dismissal of a suit for failure to exhaust remedies was vacated because the claims of discrimination had been exhausted before the equal employment opportunity commission  appellate information decided published 2019 08 28 judges manion court united states seventh circuit counsel
flores v  pompeo  18 40699 read 

aldf v  usda  18 16327 read aldf v  usda  18 16327 read affirmed  concluded that the term  individual  in the context of expedited freedom of information act requests does not include animals as well as human beings  affirmed the district court s grant of summary judgement in favor of the usda  appellate information decided published 2019 08 12 judges fletcher court united states ninth circuit counsel
in re a w   086160 read in re a w   086160 read order terminating parental rights and freeing child for adoption was reversed and remanded for determination of whether the child is a chukchansi indian  if child found to be an indian child as defined by the indian child welfare act  juvenile court ordered to proceed in accordance with icwa  appellate information decided published 2019 08 12 judges raye court california court of appeal counsel
johnson v  housing authority of city of oakland  154836 read johnson v  housing authority of city of oakland  154836 read reversed  defendant  housin

Message: no such element: Unable to locate element: {"method":"css selector","selector":".caselawcontent"}
  (Session info: chrome=85.0.4183.83)

pierre paul v  barr  18 60275 read pierre paul v  barr  18 60275 read denied in part  dismissed in part  a haitian national s petition for review of a board of immigration appeals denial of asylum and cancellation of removal was dismissed for lack of jurisdiction  his arguments that the court lacked jurisdiction and violated due process were denied  appellate information decided published 2019 07 18 judges elrod court united states fifth circuit counsel
brown v  city of sacramento  082826 read brown v  city of sacramento  082826 read affirmed  plaintiff sued employer  defendant  for racial discrimination and retaliation  a jury found in favor of plaintiff  trial court granted defendant s motion for judgment notwithstanding verdict on the grounds that plaintiff had failed to exhaust administrative remedies  but denied the motion with respect t

kisor v wilkie  18 15 read kisor v wilkie  18 15 read vacated and remanded  plaintiff is a vietnam veteran who sought disability benefits from the veterans administration for post traumatic stress  the va eventually granted benefits but only from the motion to re open his case and not from the date of the original application  court of appeals affirmed the ruling citing the deference doctrine  the us supreme court vacated the judgment and remanded to have the lower court determine if the deference doctrine applied in this case  appellate information decided published 2019 06 26 judges kagan court united states supreme court counsel
county of sonoma v gustely  153423 read county of sonoma v gustely  153423 read affirmed as modified  defendant failed to comply with an administrative order for various violations of county codes on his property  county filed suit and was awarded penalties  costs and attorney fees  but at a lower rate than amount ordered by administrative court  appeals cou

winsted v  berryhill  18 2228 read winsted v  berryhill  18 2228 read held that the social security administration did not adequately explain why it denied a man s application for disability insurance benefits and supplemental security income  the issue had to do with residual function capacity  reversed the district court s judgment and remanded to the federal agency  appellate information decided published 2019 02 08 judges brennan court united states seventh circuit counsel
gill v  us department of justice   17 16107 read gill v  us department of justice   17 16107 read upheld the federal government s so called functional standard for sharing of terrorism related information  u s  citizens who were the subjects of unfounded suspicion contended that the standard is arbitrary and capricious and also should have undergone apa notice and comment procedure  unpersuaded  the ninth circuit affirmed summary judgment for the government  appellate information decided published 2019 01 29 judg

...

lazaro v  dep t of veterans affairs  2011 3190 read lazaro v  dep t of veterans affairs  2011 3190 read on appeal of a final order of the merit systems protection board that denied the plaintiff s claim for relief under the veterans employment opportunities act  the order is vacated and the case remanded  where  1  the board had jurisdiction to determine whether the veterans administration properly afforded the plaintiff the right to compete for the job and whether it properly determined that the plaintiff was not qualified for the position  and 2  the board committed legal error by concluding that the administrative judge properly determined that the board lacked jurisdiction over the plaintiff s claim and that the administrative judge s analysis was not erroneous  appellate information decided 02 02 2012 published 02 02 2012 judges o malley court united states federal circuit counsel for appellant  douglas g  edelshick
Message: no such element: Unable to locate element: {"method":"cs

oshodi v  holder  08 71478 read oshodi v  holder  08 71478 read on a petition seeking review of the bia s denial of a request for withholding of removal and protection under the convention against torture  cat  based on the petitioner s fear of persecution and torture due to his religion and political opinions  the petition is denied  where  1  the bia did not fail to conduct an inquiry in accordance with a prior ninth circuit mandate  2  the immigration judge did not err by basing his credibility determination on the petitioner s failure to produce corroborating evidence  3  the immigration judge s adverse credibility determination was supported by substantial evidence  4  the immigration judge did not prevent the petitioner from testifying about his past torture and persecution  5  the petitioner did not overcome the presumption that the bia reviewed his evidence  6  the petitioner failed to properly authenticate certain documents by certification or by his testimony  and 7  the bia 

scibelli v  prudential insurance company  11 1372 read scibelli v  prudential insurance company  11 1372 read in an appeal from an order of the district court dismissing  on summary judgment  plaintiff s erisa claim  29 u s c  section 1132  order is vacated where based on the relevant evidence  the group policy language  and the unexplained inconsistency in the plan administrator s award of benefits  the deceased beneficiary was  totally disabled  under the terms of subject group policy when he stopped working  and therefore  his estate is entitled to the proceeds of a group life insurance policy  appellate information decided 01 11 2012 published 01 11 2012 judges lynch court united states first circuit counsel
hosanna tabor evangelical lutheran church and school v  eeoc  10 553 read hosanna tabor evangelical lutheran church and school v  eeoc  10 553 read in an appeal from a judgment of the appeals court vacating the trial court s dismissal of plaintiff s ada action on the grounds th

Message: no such element: Unable to locate element: {"method":"css selector","selector":".caselawcontent"}
  (Session info: chrome=85.0.4183.83)

cal  redevelopment assn  v  matosantos  s194861 read cal  redevelopment assn  v  matosantos  s194861 read in a writ challenge to two assembly measures intended to stabilize school funding by reducing or eliminating the diversion of property tax revenues from school districts to the state s community redevelopment agencies  petition is granted in part and denied in part where under the state constitution  1  redevelopment agencies  once created and engaged in redevelopment plans  do not have a protected right to exist that immunizes them from statutory dissolution by the legislature  and 2  redevelopment agencies and their sponsoring communities have a protected right not to make payments to various funds benefiting schools and special districts as a condition of continued operation  appellate information decided 12 29 2011 published 12 29 201

IndexError: list index out of range

In [71]:
df.to_csv('Law.csv')

In [72]:
df

,Description,Url,Date,Label,Text,Status
0,United States Seventh Circuit,https://caselaw.findlaw.com/summary/opinion/us...,08/28/2019,"Civil Procedure, Constitutional Law, Administr...",In the \nUnited States Court of Appeals For...,affirmed
1,United States Seventh Circuit,https://caselaw.findlaw.com/summary/opinion/us...,08/28/2019,"Civil Procedure, Labor & Employment Law, Admin...","United States Court of Appeals, Seventh Circui...",remanded and vacated
2,United States Fifth Circuit,https://caselaw.findlaw.com/summary/opinion/us...,08/27/2019,"Civil Procedure, Administrative Law\n ...",IN THE UNITED STATES COURT OF APPEALS\n FOR T...,affirmed and dismissed
3,United States Seventh Circuit,https://caselaw.findlaw.com/summary/opinion/us...,08/26/2019,"Civil Procedure, Administrative Law\n ...",In the\n United States Court of Appeals\n F...,affirmed
4,California Court of Appeal,https://caselaw.findlaw.com/summary/opinion/ca...,08/26/2019,"Juvenile Law, Administrative Law\n ...","In re Nicole S., 154443\nRead In re Nicole S.,...",affirmed and held
...,...,...,...,...,...,...
884,United States Federal Circuit,https://caselaw.findlaw.com/summary/opinion/us...,01/03/2012,"Administrative Law, Government Benefits, Healt...","United States Court of Appeals,Federal Circuit...",affirmed
885,United States Second Circuit,https://caselaw.findlaw.com/summary/opinion/us...,12/29/2011,"Administrative Law, Habeas Corpus, Family Law,...","United States Court of Appeals,Second Circuit....",remanded and vacated
886,United States Ninth Circuit,https://caselaw.findlaw.com/summary/opinion/us...,12/29/2011,"Administrative Law, Civil Procedure, Construct...","United States Court of Appeals,Ninth Circuit.\...",affirmed and reversed
887,Supreme Court of California,https://caselaw.findlaw.com/summary/opinion/ca...,12/29/2011,"Administrative Law, Civil Procedure, Education...",,denied


In [77]:
df.Text.drop_duplicates()

0         In the \nUnited States Court of Appeals For...
1      United States Court of Appeals, Seventh Circui...
2       IN THE UNITED STATES COURT OF APPEALS\n FOR T...
3         In the\n United States Court of Appeals\n F...
4      In re Nicole S., 154443\nRead In re Nicole S.,...
                             ...                        
883    United States Court of Appeals,Third Circuit.\...
884    United States Court of Appeals,Federal Circuit...
885    United States Court of Appeals,Second Circuit....
886    United States Court of Appeals,Ninth Circuit.\...
888    United States Court of Appeals,Federal Circuit...
Name: Text, Length: 507, dtype: object

In [ ]:
# chrome_options = webdriver.ChromeOptions()
# prefs = {"profile.default_content_setting_values.notifications" : 2}
# chrome_options.add_experimental_option("prefs",prefs)
# browser = webdriver.Chrome('./chromedriver',chrome_options=chrome_options)

# URL='https://caselaw.findlaw.com/summary/search'

# browser.get(URL)
# browser.maximize_window()
# time.sleep(sleep_time)
# browser.execute_script('window.scrollTo(0, document.body.scrollHeight / 4)')
# time.sleep(sleep_time)
# select=Select(browser.find_element_by_name('topic'))
# time.sleep(sleep_time)
# select.select_by_value('cs_1')
# time.sleep(sleep_time)
# searchbtn=browser.find_element_by_class_name('primary')
# searchbtn.click()
# time.sleep(sleep_time)
# ttlbtn=browser.find_element_by_link_text('Lockett v. Bonson').click()
# time.sleep(sleep_time)
# tinybtn=browser.find_element_by_link_text('READ').click()
# time.sleep(sleep_time)
# herebtn=browser.find_element_by_link_text('here')
# herebtn.click()
# browser.switch_to.window(browser.window_handles[1])
# url=browser.current_url
# file = urllib.request.urlopen(url)
# pdffile = open('Lawfile.pdf', 'wb')
# pdffile.write(file.read())
# file.close()
# browser.quit()